In [1]:
import numpy as np
import pandas as pd

In [2]:
ac=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/실제화물.csv',encoding='cp949')

In [3]:
rc=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/추천화물.csv',encoding='cp949')

In [4]:
ac

,Driver,Cargo
0,4,"[119099, 957508, 957471]"
1,6,"[970366, 1028529]"
2,12,[141758]
3,22,"[139338, 146412, 561963, 562222, 760384, 1018933]"
4,45,[520151]
...,...,...
15327,99995,"[702904, 833650, 968319]"
15328,99998,"[169910, 213450, 471035, 494979, 518844, 54558..."
15329,99999,"[171149, 636294]"
15330,33431833,[825527]


In [5]:
rc

,Driver,Cargo
0,4,247219
1,4,247399
2,4,242427
3,4,243372
4,4,241699
...,...,...
459955,40217675,177566
459956,40217675,171601
459957,40217675,173149
459958,40217675,197239


In [6]:
import ast  # 안전한 문자열 파싱을 위해 ast 모듈 사용

# 'Cargo' 컬럼의 데이터가 문자열 리스트라면, 이를 실제 리스트로 변환
ac['Cargo'] = ac['Cargo'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 수정된 코드를 사용하여 'Match' 컬럼 계산
for index, rc_row in rc.iterrows():
    # 실제 화물 데이터프레임에서 현재 차주에 해당하는 행 찾기
    ac_row = ac[ac['Driver'] == rc_row['Driver']]
    if not ac_row.empty:
        # 실제 화물 리스트 추출
        actual_cargos_list = ac_row.iloc[0]['Cargo']
        # 추천 화물이 실제 화물 리스트에 포함되어 있는지 검사
        if rc_row['Cargo'] in actual_cargos_list:
            rc.at[index, 'Match'] = 1

# 결과 확인
print(rc.head())


   Driver   Cargo  Match
0       4  247219    NaN
1       4  247399    NaN
2       4  242427    NaN
3       4  243372    NaN
4       4  241699    NaN


In [7]:
rc

,Driver,Cargo,Match
0,4,247219,NaN
1,4,247399,NaN
2,4,242427,NaN
3,4,243372,NaN
4,4,241699,NaN
...,...,...,...
459955,40217675,177566,NaN
459956,40217675,171601,NaN
459957,40217675,173149,NaN
459958,40217675,197239,NaN


In [8]:
import pandas as pd

# 가정: rc는 이미 'Match' 컬럼이 계산된 상태
# 각 차주별 실제 선택한 화물의 수를 계산
total_actual_cargos = ac.explode('Cargo').groupby('Driver').size()

# K 값 설정
K_values = [10, 20, 30]

# 각 K 값에 대한 Recall과 Precision 계산
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    rc_sorted = rc.sort_values(by=['Driver', 'Match'], ascending=[True, False])
    rc_top_k = rc_sorted.groupby('Driver').head(K)
    total_recommended_cargos = rc_top_k.groupby('Driver').size()
    total_matched_cargos = rc_top_k[rc_top_k['Match'] == 1].groupby('Driver').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력
results_df = pd.DataFrame(results)
results_df


,K,Recall,Precision
0,10,0.000187,0.000215
1,20,0.000187,0.000108
2,30,0.000187,0.000072


In [9]:
# 각 K 값에 대한 Recall과 Precision 계산 결과를 저장할 리스트 초기화
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    rc_sorted = rc.sort_values(by=['Driver', 'Match'], ascending=[True, False])
    rc_top_k = rc_sorted.groupby('Driver').head(K)
    total_recommended_cargos = rc_top_k.groupby('Driver').size()
    total_matched_cargos = rc_top_k[rc_top_k['Match'] == 1].groupby('Driver').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력, 소수점 10자리까지 표현
results_df = pd.DataFrame(results)
results_df['Recall'] = results_df['Recall'].apply(lambda x: round(x, 10))
results_df['Precision'] = results_df['Precision'].apply(lambda x: round(x, 10))

results_df


,K,Recall,Precision
0,10,0.087076,0.100000
1,20,0.087076,0.050000
2,30,0.087076,0.033333


In [10]:
# 차주별로 AP 계산
aps = []  # 각 차주별 Average Precision을 저장할 리스트

# 차주별로 루프
for driver in rc['Driver'].unique():
    driver_recs = rc[rc['Driver'] == driver]  # 현재 차주에 대한 추천 화물
    driver_recs_sorted = driver_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 화물별로 루프
    for i, row in enumerate(driver_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps)
print("mAP: ", map_score)


mAP:  0.002152361074876076


In [11]:
# 화물별로 AP 계산
aps = []  # 각 화물별 Average Precision을 저장할 리스트

# 화물별로 루프
for cargo in rc['Cargo'].unique():
    cargo_recs = rc[rc['Cargo'] == cargo]  # 현재 화물에 대한 추천 결과
    cargo_recs_sorted = cargo_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 차주별로 루프
    for i, row in enumerate(cargo_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps) if aps else 0
print("화물 중심 mAP: ", map_score)


화물 중심 mAP:  0.39285714285714285


In [37]:
num_matches = rc[rc['Match'] == 1].shape[0]
print(num_matches)


33


# Word2vec 성능

In [12]:
wc=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/word2vec_추천화물.csv',encoding='cp949')
wc

,Driver,Cargo,cosine,rank
0,56129,947973,0.278829,1
1,56129,881213,0.274529,2
2,56129,884111,0.260324,3
3,56129,750178,0.253518,4
4,56129,884869,0.248656,5
...,...,...,...,...
459955,89302,902060,0.247680,26
459956,89302,787098,0.247632,27
459957,89302,786598,0.247632,28
459958,89302,141758,0.246338,29


In [13]:
wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459960 entries, 0 to 459959
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Driver  459960 non-null  int64  
 1   Cargo   459960 non-null  int64  
 2   cosine  459960 non-null  float64
 3   rank    459960 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 14.0 MB


In [14]:
import ast  # 안전한 문자열 파싱을 위해 ast 모듈 사용

# 'Cargo' 컬럼의 데이터가 문자열 리스트라면, 이를 실제 리스트로 변환
ac['Cargo'] = ac['Cargo'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 수정된 코드를 사용하여 'Match' 컬럼 계산
for index, wc_row in wc.iterrows():
    # 실제 화물 데이터프레임에서 현재 차주에 해당하는 행 찾기
    ac_row = ac[ac['Driver'] == wc_row['Driver']]
    if not ac_row.empty:
        # 실제 화물 리스트 추출
        actual_cargos_list = ac_row.iloc[0]['Cargo']
        # 추천 화물이 실제 화물 리스트에 포함되어 있는지 검사
        if wc_row['Cargo'] in actual_cargos_list:
            wc.at[index, 'Match'] = 1

# 결과 확인
print(wc.head())


   Driver   Cargo    cosine  rank  Match
0   56129  947973  0.278829     1    NaN
1   56129  881213  0.274529     2    NaN
2   56129  884111  0.260324     3    NaN
3   56129  750178  0.253518     4    NaN
4   56129  884869  0.248656     5    NaN


In [15]:
wc

,Driver,Cargo,cosine,rank,Match
0,56129,947973,0.278829,1,NaN
1,56129,881213,0.274529,2,NaN
2,56129,884111,0.260324,3,NaN
3,56129,750178,0.253518,4,NaN
4,56129,884869,0.248656,5,NaN
...,...,...,...,...,...
459955,89302,902060,0.247680,26,NaN
459956,89302,787098,0.247632,27,NaN
459957,89302,786598,0.247632,28,NaN
459958,89302,141758,0.246338,29,NaN


In [16]:
wc['Match'] = wc['Match'].fillna(0)
wc

,Driver,Cargo,cosine,rank,Match
0,56129,947973,0.278829,1,0.0
1,56129,881213,0.274529,2,0.0
2,56129,884111,0.260324,3,0.0
3,56129,750178,0.253518,4,0.0
4,56129,884869,0.248656,5,0.0
...,...,...,...,...,...
459955,89302,902060,0.247680,26,0.0
459956,89302,787098,0.247632,27,0.0
459957,89302,786598,0.247632,28,0.0
459958,89302,141758,0.246338,29,0.0


In [17]:
num_matches = wc[wc['Match'] == 1].shape[0]
print(num_matches)


30


In [18]:
import pandas as pd

# 가정: rc는 이미 'Match' 컬럼이 계산된 상태
# 각 차주별 실제 선택한 화물의 수를 계산
total_actual_cargos = ac.explode('Cargo').groupby('Driver').size()

# K 값 설정
K_values = [10, 20, 30]

# 각 K 값에 대한 Recall과 Precision 계산
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    wc_sorted = wc.sort_values(by=['Cargo', 'Match'], ascending=[True, False])
    wc_top_k = wc_sorted.groupby('Cargo').head(K)
    total_recommended_cargos = wc_top_k.groupby('Cargo').size()
    total_matched_cargos = wc_top_k[wc_top_k['Match'] == 1].groupby('Cargo').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력
results_df = pd.DataFrame(results)
results_df


,K,Recall,Precision
0,10,0.0,0.008646
1,20,0.0,0.004323
2,30,0.0,0.002882


In [19]:
# 차주별로 AP 계산
aps = []  # 각 차주별 Average Precision을 저장할 리스트

# 차주별로 루프
for driver in wc['Driver'].unique():
    driver_recs = wc[rc['Driver'] == driver]  # 현재 차주에 대한 추천 화물
    driver_recs_sorted = driver_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 화물별로 루프
    for i, row in enumerate(driver_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps)
print("mAP: ", map_score)


mAP:  0.0016305765718758153


In [20]:
# 각 K 값에 대한 Recall과 Precision 계산 결과를 저장할 리스트 초기화
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    wc_sorted = wc.sort_values(by=['Driver', 'Match'], ascending=[True, False])
    wc_top_k = wc_sorted.groupby('Driver').head(K)
    total_recommended_cargos = wc_top_k.groupby('Driver').size()
    total_matched_cargos = wc_top_k[wc_top_k['Match'] == 1].groupby('Driver').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력, 소수점 10자리까지 표현
results_df = pd.DataFrame(results)
results_df['Recall'] = results_df['Recall'].apply(lambda x: round(x, 10))
results_df['Precision'] = results_df['Precision'].apply(lambda x: round(x, 10))

results_df


,K,Recall,Precision
0,10,0.155815,0.12
1,20,0.155815,0.06
2,30,0.155815,0.04


In [21]:
# 화물별로 AP 계산
aps = []  # 각 화물별 Average Precision을 저장할 리스트

# 화물별로 루프
for cargo in wc['Cargo'].unique():
    cargo_recs = wc[wc['Cargo'] == cargo]  # 현재 화물에 대한 추천 결과
    cargo_recs_sorted = cargo_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 차주별로 루프
    for i, row in enumerate(cargo_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps) if aps else 0
print("화물 중심 mAP: ", map_score)


화물 중심 mAP:  0.08645533141210375


# 오토인코더 성능평가

In [22]:
auto=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/오토인코더_추천화물.csv',encoding='cp949')
auto

,Driver,Cargo
0,56129.0,"[165859.0, 165907.0, 130474.0, 130214.0, 13238..."
1,85477.0,"[165859.0, 165907.0, 130474.0, 130214.0, 13238..."
2,22569.0,"[165859.0, 165907.0, 130474.0, 130214.0, 13238..."
3,84744.0,"[258656.0, 324157.0, 324470.0, 760802.0, 76088..."
4,21957.0,"[258656.0, 324157.0, 324470.0, 760802.0, 76088..."
...,...,...
15327,98442.0,"[1042030.0, 711472.0, 683817.0, 169816.0, 5607..."
15328,86483.0,"[165859.0, 165907.0, 130474.0, 130214.0, 13238..."
15329,87194.0,"[191526.0, 213857.0, 324470.0, 324157.0, 34031..."
15330,21573.0,"[165859.0, 165907.0, 130474.0, 130214.0, 13238..."


In [23]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15332 entries, 0 to 15331
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Driver  15332 non-null  float64
 1   Cargo   15332 non-null  object 
dtypes: float64(1), object(1)
memory usage: 239.7+ KB


In [24]:
import ast

# 'Cargo' 컬럼의 문자열을 파이썬 리스트로 변환한 후 각 원소를 정수로 변환
auto['Cargo'] = auto['Cargo'].apply(lambda x: [int(item) for item in ast.literal_eval(x)])

# 결과 확인
print(auto.head())


    Driver                                              Cargo
0  56129.0  [165859, 165907, 130474, 130214, 132380, 16981...
1  85477.0  [165859, 165907, 130474, 130214, 132380, 16981...
2  22569.0  [165859, 165907, 130474, 130214, 132380, 16981...
3  84744.0  [258656, 324157, 324470, 760802, 760889, 76081...
4  21957.0  [258656, 324157, 324470, 760802, 760889, 76081...


In [25]:
# 'Driver' 컬럼을 정수로 변환
auto['Driver'] = auto['Driver'].astype(int)

# 'Cargo' 리스트의 각 원소를 정수로 변환
auto['Cargo'] = auto['Cargo'].apply(lambda x: [int(item) for item in x])

# 결과 확인
print(auto)

       Driver                                              Cargo
0       56129  [165859, 165907, 130474, 130214, 132380, 16981...
1       85477  [165859, 165907, 130474, 130214, 132380, 16981...
2       22569  [165859, 165907, 130474, 130214, 132380, 16981...
3       84744  [258656, 324157, 324470, 760802, 760889, 76081...
4       21957  [258656, 324157, 324470, 760802, 760889, 76081...
...       ...                                                ...
15327   98442  [1042030, 711472, 683817, 169816, 560740, 5459...
15328   86483  [165859, 165907, 130474, 130214, 132380, 16981...
15329   87194  [191526, 213857, 324470, 324157, 340316, 25865...
15330   21573  [165859, 165907, 130474, 130214, 132380, 16981...
15331   89302  [165859, 165907, 130474, 130214, 132380, 16981...

[15332 rows x 2 columns]


In [26]:
# 예시: 'Cargo' 컬럼을 explode하여 각 화물을 별도의 행으로 확장
auto_exploded = auto.explode('Cargo')
auto_exploded['Cargo'] = auto_exploded['Cargo'].astype('int64')
auto_exploded

,Driver,Cargo
0,56129,165859
0,56129,165907
0,56129,130474
0,56129,130214
0,56129,132380
...,...,...
15331,89302,242652
15331,89302,243677
15331,89302,313418
15331,89302,250181


In [27]:
auto_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459960 entries, 0 to 15331
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   Driver  459960 non-null  int32
 1   Cargo   459960 non-null  int64
dtypes: int32(1), int64(1)
memory usage: 8.8 MB


In [28]:
# 'Match' 컬럼을 0으로 초기화
auto_exploded['Match'] = 0

# auto_exploded 데이터프레임에 대해 iterrows() 대신 apply() 사용
def check_match(row):
    # 실제 화물 데이터프레임에서 현재 차주에 해당하는 행 찾기
    ac_row = ac[ac['Driver'] == row['Driver']]
    if not ac_row.empty:
        # 실제 화물 리스트 추출
        actual_cargos_list = ac_row.iloc[0]['Cargo']
        # 현재 화물이 실제 화물 리스트에 포함되어 있는지 검사
        return 1 if row['Cargo'] in actual_cargos_list else 0
    else:
        return 0

# 각 행에 대해 check_match 함수 적용
auto_exploded['Match'] = auto_exploded.apply(check_match, axis=1)

# 결과 확인
print(auto_exploded.head())


   Driver   Cargo  Match
0   56129  165859      0
0   56129  165907      0
0   56129  130474      0
0   56129  130214      0
0   56129  132380      0


In [29]:
auto = auto_exploded

In [30]:
# auto.to_csv("auto.csv", index=False)

In [31]:
num_matches = auto[auto['Match'] == 1].shape[0]
print(num_matches)


27


In [32]:
import pandas as pd

# 가정: rc는 이미 'Match' 컬럼이 계산된 상태
# 각 차주별 실제 선택한 화물의 수를 계산
total_actual_cargos = ac.explode('Cargo').groupby('Driver').size()

# K 값 설정
K_values = [10, 20, 30]

# 각 K 값에 대한 Recall과 Precision 계산
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    auto_sorted = auto.sort_values(by=['Driver', 'Match'], ascending=[True, False])
    auto_top_k = auto_sorted.groupby('Driver').head(K)
    total_recommended_cargos = auto_top_k.groupby('Driver').size()
    total_matched_cargos = auto_top_k[auto_top_k['Match'] == 1].groupby('Driver').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력
results_df = pd.DataFrame(results)
results_df


,K,Recall,Precision
0,10,0.000193,0.000176
1,20,0.000193,0.000088
2,30,0.000193,0.000059


In [33]:
# 각 K 값에 대한 Recall과 Precision 계산 결과를 저장할 리스트 초기화
results = []

for K in K_values:
    # 각 차주별로 상위 K개의 추천 화물을 고려하여 매치된 화물의 수를 계산
    auto_sorted = auto.sort_values(by=['Driver', 'Match'], ascending=[True, False])
    auto_top_k = auto_sorted.groupby('Driver').head(K)
    total_recommended_cargos = auto_top_k.groupby('Driver').size()
    total_matched_cargos = auto_top_k[auto_top_k['Match'] == 1].groupby('Driver').size()

    # Recall과 Precision 계산
    recall = (total_matched_cargos / total_actual_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)
    precision = (total_matched_cargos / total_recommended_cargos.reindex(total_matched_cargos.index, fill_value=0)).fillna(0)

    results.append({
        'K': K,
        'Recall': recall.mean(),
        'Precision': precision.mean()
    })

# 결과를 DataFrame으로 변환하여 출력, 소수점 10자리까지 표현
results_df = pd.DataFrame(results)
results_df['Recall'] = results_df['Recall'].apply(lambda x: round(x, 10))
results_df['Precision'] = results_df['Precision'].apply(lambda x: round(x, 10))

results_df


,K,Recall,Precision
0,10,0.113743,0.103846
1,20,0.113743,0.051923
2,30,0.113743,0.034615


In [34]:
# 차주별로 AP 계산
aps = []  # 각 차주별 Average Precision을 저장할 리스트

# 차주별로 루프
for driver in auto['Driver'].unique():
    driver_recs = auto[auto['Driver'] == driver]  # 현재 차주에 대한 추천 화물
    driver_recs_sorted = driver_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 화물별로 루프
    for i, row in enumerate(driver_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps)
print("mAP: ", map_score)


mAP:  0.0016957996347508478


In [35]:
# 화물별로 AP 계산
aps = []  # 각 화물별 Average Precision을 저장할 리스트

# 화물별로 루프
for cargo in auto['Cargo'].unique():
    cargo_recs = auto[auto['Cargo'] == cargo]  # 현재 화물에 대한 추천 결과
    cargo_recs_sorted = cargo_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 차주별로 루프
    for i, row in enumerate(cargo_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps) if aps else 0
print("화물 중심 mAP: ", map_score)


화물 중심 mAP:  0.01828029790115098


In [36]:
# 'Cargo' 컬럼을 펼쳐서 각 화물이 별도의 행을 갖도록 함
exploded_auto = auto.explode('Cargo')

# 화물별로 AP 계산
aps = []  # 각 화물별 Average Precision을 저장할 리스트

# 펼쳐진 데이터프레임에서 화물별로 루프
for cargo in exploded_auto['Cargo'].unique():
    cargo_recs = exploded_auto[exploded_auto['Cargo'] == cargo]
    cargo_recs_sorted = cargo_recs.sort_values(by='Match', ascending=False)
    
    cum_precisions = []  # 누적 정밀도를 저장할 리스트
    relevant_items_count = 0  # 관련 항목의 수
    
    # 추천된 화물별로 루프
    for i, row in enumerate(cargo_recs_sorted.itertuples(), 1):
        if row.Match == 1:  # 관련 항목이면
            relevant_items_count += 1
            precision_at_i = relevant_items_count / i
            cum_precisions.append(precision_at_i)
    
    # AP 계산
    ap = sum(cum_precisions) / relevant_items_count if relevant_items_count > 0 else 0
    aps.append(ap)

# mAP 계산
map_score = sum(aps) / len(aps) if aps else 0
print("화물 중심 mAP: ", map_score)


화물 중심 mAP:  0.01828029790115098


In [38]:
matches_data = rc[rc['Match'] == 1]

# 필터링된 데이터 출력
print(matches_data)

        Driver   Cargo  Match
9683      2392  129297    1.0
23577     4869  173149    1.0
29753     5717  118622    1.0
31357     5931  236953    1.0
33491     6242  206893    1.0
39669     7501  208478    1.0
71106    22118  235373    1.0
72940    22259  118608    1.0
82878    40226  135963    1.0
98579    41644  195389    1.0
116507   43942  146136    1.0
118064   44073  203503    1.0
125901   44768  120256    1.0
128855   45030  238708    1.0
146340   46699  247219    1.0
208366   51712  144583    1.0
213151   52060  247399    1.0
222710   52683  120934    1.0
228103   53055  215377    1.0
234685   53485  177566    1.0
274302   55772  218116    1.0
278582   56008  241699    1.0
280521   56129  102114    1.0
315626   83844  171601    1.0
333465   86094  123340    1.0
335482   86354  122926    1.0
356728   89566  118562    1.0
383235   92559  202228    1.0
392883   93490  243372    1.0
425253   96602  242427    1.0
437590   97802  205557    1.0
444388   98499  197239    1.0
450589   9

In [39]:
matches_data = wc[wc['Match'] == 1]

# 필터링된 데이터 출력
print(matches_data)

          Driver    Cargo    cosine  rank  Match
8501           4   957508  0.233451    12    1.0
17355      97133  1150884  0.222346    16    1.0
92585      55537   736630  0.240766     6    1.0
116334  40217675   391883  0.229711    25    1.0
121048     83405   550370  0.240523    29    1.0
133700        22   139338  0.259095    21    1.0
149229     43660  1042540  0.233889    10    1.0
156613        50   761431  0.240928    14    1.0
156616        50   787098  0.239900    17    1.0
156617        50   786598  0.239900    18    1.0
156618        50   902060  0.239335    19    1.0
156623        50  1014673  0.237853    24    1.0
156624        50  1018453  0.237853    25    1.0
160866      4715   177959  0.281072     7    1.0
160891       486   295795  0.297988     2    1.0
164761     44899   947973  0.242538     2    1.0
170682     91432   530010  0.203547    13    1.0
174267     47739  1165343  0.224972    28    1.0
176364      3654  1213580  0.231895    25    1.0
176990     98420   5

In [41]:
matches_data = auto[auto['Match'] == 1]

# 필터링된 데이터 출력
print(matches_data)

       Driver   Cargo  Match
233     52152  327455      1
438      7466  314718      1
568     52838  516643      1
869     54963  569449      1
941     55056  165859      1
1261    43624  476092      1
1261    43624  376327      1
2306    91701  303418      1
2318    41340  326353      1
2413    47905  128793      1
2599    45041  141137      1
2711    44633  391370      1
2875    97888  485899      1
3012    94140  130214      1
3259    50139  643653      1
3294    54451  255103      1
3444    93826  260072      1
3756    43265  372411      1
4937    91912  897445      1
7091    81869  733994      1
7500    92611  530361      1
7844    90889  208069      1
8026    95641  211482      1
8206    52627  216697      1
9825    83665  741850      1
10083   94720  298525      1
12047   55635  473303      1
